In [1]:
# import packages
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline

import nltk
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer 
from nltk.stem.snowball import SnowballStemmer
from nltk.stem.wordnet import WordNetLemmatizer

import re

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\stdg\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\stdg\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [3]:
# load the train and test data
import os
#prepd data
train_path = os.path.expanduser('./train_test_data_20230912/train_data_prepd_20231017.csv')
test_path = os.path.expanduser('./train_test_data_20230912/test_data_prepd_20231017.xlsx')

#read test_data_en.csv to a dataframe test_data
# test_data = pd.read_csv(test_path, sep=',', encoding='utf-8')
test_data = pd.read_excel(test_path,sheet_name='Sheet1')


#drop columns "Unnamed: 0", "sentence"
# test_data=test_data.drop(['Unnamed: 0', 'sentence_en', '__index_level_0__'], axis=1)
# test_data=test_data.drop(['Unnamed: 0'], axis=1)
#rename column "sentence_en" to "sentence"
# test_data=test_data.rename(columns={'sentence_en': 'sentence'})
#read train_data_en.csv to a dataframe train_data
train_data = pd.read_csv(train_path, sep=',', encoding='utf-8')
#change the label column to int
train_data['label'] = train_data['label'].astype(int)
#drop columns "Unnamed: 0", "sentence"
# train_data=train_data.drop(['Unnamed: 0', 'sentence'], axis=1)
#rename column "sentence_en" to "sentence"
# train_data=train_data.rename(columns={'sentence_en': 'sentence'})
#print column names of train_data
print(train_data.columns)
#print count values of label column in train_data
print(pd.crosstab(index=train_data['label'], columns='count'))
#print column names of test_data
print(test_data.columns)
#print count values of label column in test_data
print(pd.crosstab(index=test_data['label'], columns='count'))
#print first 10 rows of test_data sentence column
# print(test_data['sentence'].head(10))
#print first 10 rows of train_data sentence column
# print(train_data['sentence'].head(10))


#drop all empty rows in test_data
test_data=test_data.dropna()

Index(['label', 'sentence'], dtype='object')
col_0  count
label       
0        689
1        915
2        773
3        226
4       1235
Index(['label', 'sentence'], dtype='object')
col_0  count
label       
0         78
1         72
2         97
3         81
4        194


In [4]:
#load excel table test_data.xlsx as pandas dataframe
# train_data = pd.read_excel('train_data.xlsx', sheet_name='Sheet1')
#print label counts 
print(train_data.label.value_counts())
#load excel table test_data.xlsx as pandas dataframe
# test_data = pd.read_excel('test_data.xlsx', sheet_name='Sheet1')
print(test_data.label.value_counts())

4    1235
1     915
2     773
0     689
3     226
Name: label, dtype: int64
4    194
2     97
3     81
0     78
1     72
Name: label, dtype: int64


In [5]:
# #remove rows with column label 4 from test_data and train_data
# label_4=test_data['label']!=4
# test_data=test_data[label_4]
# label_4=train_data['label']!=4
# train_data=train_data[label_4]

# train_data=train_data.dropna()
# test_data=test_data.dropna()

# #print label values count for dataframe test_data and train_data
# #print commentary "alle test daten label frequenz"
# print("alle test daten label frequenz")
# print(pd.crosstab(index=test_data['label'], columns='count'))
# #print commentary "alle train daten label frequenz"
# print("alle train daten label frequenz")
# print(pd.crosstab(index=train_data['label'], columns='count'))



train = train_data[['label','sentence']].copy()
test = test_data[['label','sentence']].copy()

### Preprocessing the dataset begins here

In [7]:
all_corpus = pd.Series(train.sentence.tolist() + test.sentence.tolist()).astype(str)
# all_corpus

In [8]:
def text_clean(corpus):
    '''
    Purpose : Function to keep only alphabets, digits and certain words (punctuations, qmarks, tabs etc. removed)
    
    Input : Takes a text corpus, 'corpus' to be cleaned along with a list of words, 'keep_list', which have to be retained
            even after the cleaning process
    
    Output : Returns the cleaned text corpus
    
    '''
    cleaned_corpus = pd.Series(dtype='object')
    for row in corpus:
        qs = []
        for word in row.split():
            p1 = re.sub(pattern='[^a-zA-Z]',repl=' ',string=word)
            p1 = p1.lower()
            qs.append(p1)
        cleaned_corpus = pd.concat([cleaned_corpus, pd.Series(' '.join(qs))])    
        # cleaned_corpus = cleaned_corpus.append(pd.Series(' '.join(qs)))
    return cleaned_corpus
import re
import pickle
import sklearn
import pandas as pd
import numpy as np
import holoviews as hv
import hvplot
import nltk 
from bokeh.io import output_notebook
output_notebook()

from hvplot import pandas
from pathlib import Path

#hv.extension("bokeh")

pd.options.display.max_columns = 100
pd.options.display.max_rows = 300
pd.options.display.max_colwidth = 100
np.set_printoptions(threshold=2000)

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
#from xgboost import XGBClassifier
from nltk.stem.snowball import SnowballStemmer
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

nltk.download('punkt')
nltk.download('stopwords')

stemmer = SnowballStemmer("german")
stop_words = set(stopwords.words("german"))


def clean_text(text, for_embedding=False):
    """
        - remove any html tags (< /br> often found)
        - Keep only ASCII + European Chars and whitespace, no digits
        - remove single letter chars
        - convert all whitespaces (tabs etc.) to single wspace
        if not for embedding (but e.g. tdf-idf):
        - all lowercase
        - remove stopwords, punctuation and stemm
    """
    RE_WSPACE = re.compile(r"\s+", re.IGNORECASE)
    RE_TAGS = re.compile(r"<[^>]+>")
    RE_ASCII = re.compile(r"[^A-Za-zÀ-ž ]", re.IGNORECASE)
    RE_SINGLECHAR = re.compile(r"\b[A-Za-zÀ-ž]\b", re.IGNORECASE)
    if for_embedding:
        # Keep punctuation
        RE_ASCII = re.compile(r"[^A-Za-zÀ-ž,.!? ]", re.IGNORECASE)
        RE_SINGLECHAR = re.compile(r"\b[A-Za-zÀ-ž,.!?]\b", re.IGNORECASE)

    text = re.sub(RE_TAGS, " ", text)
    text = re.sub(RE_ASCII, " ", text)
    text = re.sub(RE_SINGLECHAR, " ", text)
    text = re.sub(RE_WSPACE, " ", text)

    word_tokens = word_tokenize(text)
    words_tokens_lower = [word.lower() for word in word_tokens]

    if for_embedding:
        # no stemming, lowering and punctuation / stop words removal
        words_filtered = word_tokens
    else:
        words_filtered = [
            stemmer.stem(word) for word in words_tokens_lower if word not in stop_words
        ]

    text_clean = " ".join(words_filtered)
    return text_clean
def stopwords_removal(corpus):
    wh_words = ['who', 'what', 'when', 'why', 'how', 'which', 'where', 'whom']
    stop = set(stopwords.words('german'))
    for word in wh_words:
        stop.remove(word)
    corpus = [[x for x in x.split() if x not in stop] for x in corpus]
    return corpus
def lemmatize(corpus):
    lem = WordNetLemmatizer()
    corpus = [[lem.lemmatize(x, pos = 'v') for x in x] for x in corpus]
    return corpus
def stem(corpus, stem_type = None):
    if stem_type == 'snowball':
        stemmer = SnowballStemmer(language = 'german')
        corpus = [[stemmer.stem(x) for x in x] for x in corpus]
    else :
        stemmer = PorterStemmer()
        corpus = [[stemmer.stem(x) for x in x] for x in corpus]
    return corpus
def preprocess(corpus, cleaning = True, stemming = False, stem_type = None, lemmatization = False, remove_stopwords = True):
    
    '''
    Purpose : Function to perform all pre-processing tasks (cleaning, stemming, lemmatization, stopwords removal etc.)
    
    Input : 
    'corpus' - Text corpus on which pre-processing tasks will be performed
    
    'cleaning', 'stemming', 'lemmatization', 'remove_stopwords' - Boolean variables indicating whether a particular task should 
                                                                  be performed or not
    'stem_type' - Choose between Porter stemmer or Snowball(Porter2) stemmer. Default is "None", which corresponds to Porter
                  Stemmer. 'snowball' corresponds to Snowball Stemmer
    
    Note : Either stemming or lemmatization should be used. There's no benefit of using both of them together
    
    Output : Returns the processed text corpus
    
    '''
    if cleaning == True:
        corpus = text_clean(corpus)
    
    if remove_stopwords == True:
        corpus = stopwords_removal(corpus)
    else :
        corpus = [[x for x in x.split()] for x in corpus]
    
    if lemmatization == True:
        corpus = lemmatize(corpus)
        
        
    if stemming == True:
        corpus = stem(corpus, stem_type)
    
    corpus = [' '.join(x) for x in corpus]
        

    return corpus
clean_text("Python ist die beste Programmiersprache der Welt.")

Loading BokehJS ...

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\stdg\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\stdg\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


'python best programmiersprach welt'

In [9]:
%%time
# Clean Comments
#data["comment_clean"] = data.loc[data["comment"].str.len() > 20, "comment"]
all_corpus = all_corpus.map(
    lambda x: clean_text(x, for_embedding=False) if isinstance(x, str) else x
)
all_corpus = preprocess(all_corpus, remove_stopwords = False)


Wall time: 2.28 s


In [10]:

train_corpus = all_corpus[0:train.shape[0]]
test_corpus = all_corpus[train.shape[0]:]

# Sequential Model

### TF-IDF based Vectorization of text 

In [11]:
vectorizer = TfidfVectorizer()
#tf_idf_matrix_train = vectorizer.fit_transform(train_corpus)
tf_idf_matrix_train = vectorizer.fit_transform(train_corpus).toarray()
#tf_idf_matrix_test = vectorizer.transform(test_corpus)
tf_idf_matrix_test = vectorizer.transform(test_corpus).toarray()

### We would use Keras for building our network architecture

In [12]:
# !pip install tensorflow
# !pip install keras
# !pip install np_utils
import keras 
from keras.models import Sequential, Model 
from keras import layers
from keras.layers import Dense, Dropout, Input
# import np_utils
# from keras.utils import np_utils
from keras.src.utils.np_utils import to_categorical

### Converting the Class labels into one hot encoded vectors

In [13]:
y_train = to_categorical(train['label'], train['label'].nunique())
y_test = to_categorical(test['label'], train['label'].nunique())

### Defining and Building our network architecture

In [14]:
tf_idf_matrix_train
model = Sequential()
model.add(Dense(512, activation='relu', input_dim=tf_idf_matrix_train.shape[1]))
model.add(Dropout(0.3))
model.add(Dense(5, activation='softmax'))
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 512)               9192960   
                                                                 
 dropout (Dropout)           (None, 512)               0         
                                                                 
 dense_1 (Dense)             (None, 5)                 2565      
                                                                 
Total params: 9195525 (35.08 MB)
Trainable params: 9195525 (35.08 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


### Model training

In [15]:
#tf_idf_matrix_train
#y_train
training_history = model.fit(tf_idf_matrix_train, y_train, epochs=10, batch_size=100)

Epoch 1/10
39/39 [==============================] - 2s 51ms/step - loss: 1.4739 - categorical_accuracy: 0.4039
Epoch 2/10
39/39 [==============================] - 2s 55ms/step - loss: 1.0163 - categorical_accuracy: 0.7053
Epoch 3/10
39/39 [==============================] - 2s 55ms/step - loss: 0.5419 - categorical_accuracy: 0.8950
Epoch 4/10
39/39 [==============================] - 2s 55ms/step - loss: 0.2401 - categorical_accuracy: 0.9758
Epoch 5/10
39/39 [==============================] - 2s 55ms/step - loss: 0.1108 - categorical_accuracy: 0.9937
Epoch 6/10
39/39 [==============================] - 2s 56ms/step - loss: 0.0588 - categorical_accuracy: 0.9990
Epoch 7/10
39/39 [==============================] - 2s 56ms/step - loss: 0.0360 - categorical_accuracy: 0.9997
Epoch 8/10
39/39 [==============================] - 2s 55ms/step - loss: 0.0246 - categorical_accuracy: 0.9997
Epoch 9/10
39/39 [==============================] - 2s 55ms/step - loss: 0.0180 - categorical_accuracy: 0.9995
E

### Model Evaluation on test data based on Accuracy

In [16]:
loss, accuracy = model.evaluate(tf_idf_matrix_test, y_test, verbose=False)
print("Testing Accuracy:  {:.4f}".format(accuracy))

Testing Accuracy:  0.6437


# Model Naive Bayes

In [17]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics import accuracy_score, confusion_matrix

In [18]:
y_train = train['label']
y_test = test['label']

nb = Pipeline([('vect', CountVectorizer()),
               ('tfidf', TfidfTransformer()),
               ('clf', MultinomialNB()),
              ])
nb.fit(train_corpus, y_train)


from sklearn.metrics import classification_report
y_pred = nb.predict(test_corpus)


In [19]:

my_tags = [ "Klient hat Gewalt erfahren", "Klient hat häusliche Gewalt erlebt", "Klient hat Gewalt ausgeübt",  "Klient hat sexuelle Gewalt erlebt", "Klient hat keine Gewalt erfahren"]

print('accuracy %s' % accuracy_score(y_pred, y_test))
print(classification_report(y_test, y_pred,target_names=my_tags))

accuracy 0.5210727969348659
                                    precision    recall  f1-score   support

        Klient hat Gewalt erfahren       1.00      0.08      0.14        78
Klient hat häusliche Gewalt erlebt       0.59      0.72      0.65        72
        Klient hat Gewalt ausgeübt       0.84      0.32      0.46        97
 Klient hat sexuelle Gewalt erlebt       0.00      0.00      0.00        81
  Klient hat keine Gewalt erfahren       0.47      0.94      0.63       194

                          accuracy                           0.52       522
                         macro avg       0.58      0.41      0.38       522
                      weighted avg       0.56      0.52      0.43       522



c:\Program Files\Anaconda3\envs\huggingface_env\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Program Files\Anaconda3\envs\huggingface_env\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Program Files\Anaconda3\envs\huggingface_env\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start,

# Model Linear Support Vector Machine

In [20]:
from sklearn.linear_model import SGDClassifier

sgd = Pipeline([('vect', CountVectorizer()),
                ('tfidf', TfidfTransformer()),
                ('clf', SGDClassifier(loss='hinge', penalty='l2',alpha=1e-3, random_state=42, max_iter=5, tol=None)),
               ])
sgd.fit(train_corpus, y_train)

Pipeline(steps=[('vect', CountVectorizer()), ('tfidf', TfidfTransformer()),
                ('clf',
                 SGDClassifier(alpha=0.001, max_iter=5, random_state=42,
                               tol=None))])

In [21]:
%%time

y_pred = sgd.predict(test_corpus)

print('accuracy %s' % accuracy_score(y_pred, y_test))
print(classification_report(y_test, y_pred,target_names=my_tags))

accuracy 0.6513409961685823
                                    precision    recall  f1-score   support

        Klient hat Gewalt erfahren       0.56      0.42      0.48        78
Klient hat häusliche Gewalt erlebt       0.53      0.78      0.63        72
        Klient hat Gewalt ausgeübt       0.62      0.80      0.70        97
 Klient hat sexuelle Gewalt erlebt       0.76      0.69      0.72        81
  Klient hat keine Gewalt erfahren       0.75      0.60      0.67       194

                          accuracy                           0.65       522
                         macro avg       0.64      0.66      0.64       522
                      weighted avg       0.67      0.65      0.65       522

Wall time: 16.6 ms


In [22]:
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer


In [ ]:

svm = Pipeline([
    ('vect', CountVectorizer(max_features=10000)),
    ('tfidf', TfidfTransformer()),
    ('clf', SVC(random_state=42)),
])

# Define the parameter grid
param_grid = {
    'clf__C': [0.1, 1, 10, 100],
    'clf__gamma': [1, 0.1, 0.01, 0.001],
    'clf__kernel': ['linear', 'rbf']
}

# Create a GridSearchCV object
grid_search = GridSearchCV(svm, param_grid, cv=5)

# Fit the model
grid_search.fit(train_corpus, y_train)

# Print the best parameters
print("Best parameters: ", grid_search.best_params_)

In [23]:
svm = Pipeline([
    ('vect', CountVectorizer(max_features=10000)),
    ('tfidf', TfidfTransformer()),
    ('clf', SVC(random_state=42, kernel='linear', C=1, gamma=1)),
])
svm.fit(train_corpus, y_train)

Pipeline(steps=[('vect', CountVectorizer(max_features=10000)),
                ('tfidf', TfidfTransformer()),
                ('clf', SVC(C=1, gamma=1, kernel='linear', random_state=42))])

In [24]:
%%time

y_pred = svm.predict(test_corpus)

print('accuracy %s' % accuracy_score(y_pred, y_test))
print(classification_report(y_test, y_pred,target_names=my_tags))

accuracy 0.6762452107279694
                                    precision    recall  f1-score   support

        Klient hat Gewalt erfahren       0.57      0.55      0.56        78
Klient hat häusliche Gewalt erlebt       0.62      0.74      0.68        72
        Klient hat Gewalt ausgeübt       0.63      0.77      0.69        97
 Klient hat sexuelle Gewalt erlebt       0.78      0.74      0.76        81
  Klient hat keine Gewalt erfahren       0.74      0.63      0.68       194

                          accuracy                           0.68       522
                         macro avg       0.67      0.69      0.67       522
                      weighted avg       0.68      0.68      0.68       522

Wall time: 435 ms


In [26]:
# Fit your vectorizer and transform your data
vect = CountVectorizer(max_features=10000)
tfidf = TfidfTransformer()
X_train_transformed = tfidf.fit_transform(vect.fit_transform(train_corpus))


# Define the classifier
clf = SVC(kernel='linear', C=1, random_state=42)


In [27]:

from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.svm import SVC
from sklearn.feature_selection import RFE

# Define the RFE object and compute a cross-validated score.
svc_rfe = RFE(estimator=clf, n_features_to_select=1000, step=10)
svc_rfe.fit(X_train_transformed, y_train)

# Fit the model
clf.fit(svc_rfe.transform(X_train_transformed), y_train)


SVC(C=1, kernel='linear', random_state=42)

In [28]:
%%time

#predict the labels on test dataset
y_pred = clf.predict(svc_rfe.transform(tfidf.transform(vect.transform(test_corpus))))

print('accuracy %s' % accuracy_score(y_pred, y_test))
print(classification_report(y_test, y_pred,target_names=my_tags))

accuracy 0.6590038314176245
                                    precision    recall  f1-score   support

        Klient hat Gewalt erfahren       0.57      0.53      0.55        78
Klient hat häusliche Gewalt erlebt       0.62      0.68      0.65        72
        Klient hat Gewalt ausgeübt       0.62      0.70      0.66        97
 Klient hat sexuelle Gewalt erlebt       0.76      0.77      0.76        81
  Klient hat keine Gewalt erfahren       0.69      0.64      0.66       194

                          accuracy                           0.66       522
                         macro avg       0.65      0.66      0.66       522
                      weighted avg       0.66      0.66      0.66       522

Wall time: 252 ms


# Model Doc2Vec LogReg

In [29]:
from tqdm import tqdm
tqdm.pandas(desc="progress-bar")
from gensim.models import Doc2Vec
from sklearn import utils
import gensim
from gensim.models.doc2vec import TaggedDocument
import re

def label_sentences(corpus, label_type):
    """
    Gensim's Doc2Vec implementation requires each document/paragraph to have a label associated with it.
    We do this by using the TaggedDocument method. The format will be "TRAIN_i" or "TEST_i" where "i" is
    a dummy index of the post.
    """
    labeled = []
    for i, v in enumerate(corpus):
        label = label_type + '_' + str(i)
        labeled.append(doc2vec.TaggedDocument(v.split(), [label]))
    return labeled

In [30]:
import pandas as pd
import numpy as np
from tqdm import tqdm
tqdm.pandas(desc="progress-bar")
from gensim.models import Doc2Vec
from sklearn import utils
from sklearn.model_selection import train_test_split
import gensim
from sklearn.linear_model import LogisticRegression
from gensim.models.doc2vec import TaggedDocument
import re
import seaborn as sns
import matplotlib.pyplot as plt
from gensim.models import doc2vec

In [31]:
X_train=train_data['sentence']
y_train=train_data['label']
X_test=test_data['sentence']
y_test=test_data['label']


In [32]:
%%time
# Clean Comments
#data["comment_clean"] = data.loc[data["comment"].str.len() > 20, "comment"]
X_train = X_train.map(
    lambda x: clean_text(x, for_embedding=False) if isinstance(x, str) else x
)
X_train = preprocess(X_train, remove_stopwords = False)

Wall time: 1.99 s


In [33]:
#drop all empty rows in X_test
X_test = X_test.dropna()
#drop all empty rows in y_test
y_test = y_test.dropna()
#reset index of X_test
X_test = X_test.reset_index(drop=True)
#reset index of y_test
y_test = y_test.reset_index(drop=True)

In [34]:
%%time
# Clean Comments
#data["comment_clean"] = data.loc[data["comment"].str.len() > 20, "comment"]
X_test = X_test.map(
    lambda x: clean_text(x, for_embedding=False) if isinstance(x, str) else x
)
X_test = preprocess(X_test, remove_stopwords = False)

Wall time: 284 ms


In [35]:
X_train = label_sentences(X_train, 'Train')
X_test = label_sentences(X_test, 'Test')
all_data = X_train + X_test

In [36]:
all_data[5:7]

[TaggedDocument(words=['frau', 'bockli', 'lehnt', 'vornherein', 'psychotherapi', 'ab', 'aufwand', 'ausgedehnt', 'besuchsrecht', 'bring', 'mehr', 'spannung', 'unruh', 'weiterbearbeit', 'themat', 'beid', 'elt', 'tag', 'spat', 'zeigt', 'wiederum', 'unterschied', 'einstell', 'elt', 'frau', 'bockli', 'beid', 'kind', 'geb', 'herr', 'kell', 'schlug', 'einzig', 'massnahm', 'trennung', 'kind', 'schul', 'willigt', 'vorschlag', 'gemeinsam', 'rhyhuus', 'flurling', 'besichtigendi', 'erfolgt', 'kind', 'gefiel', 'rhyhuus', 'ausgesproch', 'gut'], tags=['Train_5']),
 TaggedDocument(words=['klient', 'vat', 'geleg', 'einand', 'seh', 'regelmass', 'besuchsrecht', 'erscheint', 'jedoch', 'realist', 'hatt', 'folg', 'klient', 'imm', 'enttauscht', 'schlag', 'deshalb', 'herr', 'ruttimann', 'klient', 'zweimal', 'pro', 'jahr', 'vorhergeh', 'absprach', 'zuhaus', 'besuch', 'informationsaustausch', 'einvernehm', 'elt', 'klientfrau', 'knusel', 'herrn', 'ruttimann', 'novemb', 'februar', 'monat', 'mittel', 'brief', 'ent

In [37]:
model_dbow = Doc2Vec(dm=0, vector_size=300, negative=5, min_count=1, alpha=0.065, min_alpha=0.065)
model_dbow.build_vocab([x for x in tqdm(all_data)])

for epoch in range(30):
    model_dbow.train(utils.shuffle([x for x in tqdm(all_data)]), total_examples=len(all_data), epochs=1)
    model_dbow.alpha -= 0.002
    model_dbow.min_alpha = model_dbow.alpha

100%|██████████| 4360/4360 [00:00<00:00, 4527646.80it/s]


100%|██████████| 4360/4360 [00:00<?, ?it/s]


In [38]:
len(y_test)

522

In [39]:
def get_vectors(model, corpus_size, vectors_size, vectors_type):
    """
    Get vectors from trained doc2vec model
    :param doc2vec_model: Trained Doc2Vec model
    :param corpus_size: Size of the data
    :param vectors_size: Size of the embedding vectors
    :param vectors_type: Training or Testing vectors
    :return: list of vectors
    """
    vectors = np.zeros((corpus_size, vectors_size))
    for i in range(0, corpus_size):
        prefix = vectors_type + '_' + str(i)
        vectors[i] = model.docvecs[prefix]
    return vectors
    
train_vectors_dbow = get_vectors(model_dbow, len(X_train), 300, 'Train')
test_vectors_dbow = get_vectors(model_dbow, len(X_test), 300, 'Test')

<ipython-input-39-81a54864760a>:13: DeprecationWarning: Call to deprecated `docvecs` (The `docvecs` property has been renamed `dv`.).
  vectors[i] = model.docvecs[prefix]


In [40]:
logreg = LogisticRegression(n_jobs=1, C=1e5)
logreg.fit(train_vectors_dbow, y_train)
logreg = logreg.fit(train_vectors_dbow, y_train)


y_pred = logreg.predict(test_vectors_dbow)


print('accuracy %s' % accuracy_score(y_pred, y_test))
print(classification_report(y_test, y_pred,target_names=my_tags))

accuracy 0.6053639846743295
                                    precision    recall  f1-score   support

        Klient hat Gewalt erfahren       0.47      0.55      0.51        78
Klient hat häusliche Gewalt erlebt       0.53      0.67      0.59        72
        Klient hat Gewalt ausgeübt       0.59      0.72      0.65        97
 Klient hat sexuelle Gewalt erlebt       0.72      0.67      0.69        81
  Klient hat keine Gewalt erfahren       0.69      0.52      0.59       194

                          accuracy                           0.61       522
                         macro avg       0.60      0.63      0.61       522
                      weighted avg       0.62      0.61      0.61       522



c:\Program Files\Anaconda3\envs\huggingface_env\lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
c:\Program Files\Anaconda3\envs\huggingface_env\lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/l